In [1]:
collab=1

In [2]:
if collab:
  from google.colab import drive
  drive.mount('/content/drive')

  %cd /content/drive/MyDrive/Projects/GO_GO_Nihongo

Mounted at /content/drive
/content/drive/MyDrive/Projects/GO_GO_Nihongo


In [3]:
from IPython.display import clear_output

%pip install datasets
%pip install janome mojimoji
%pip install transformers
%pip install sentencepiece
%pip install tensorflow
clear_output()

In [4]:
from janome.tokenizer import Tokenizer
import mojimoji

# Initialize Japanese tokenizer
tokenizer = Tokenizer()


def preprocess_japanese_text(text):
    # Convert full-width characters to half-width
    text = mojimoji.zen_to_han(text)

    # Tokenize the text
    tokens = tokenizer.tokenize(text, wakati=True)

    # Join tokens with a space to preserve word boundaries for translation models
    preprocessed_text = " ".join(tokens)

    # Remove any unnecessary whitespace
    preprocessed_text = preprocessed_text.strip()

    return preprocessed_text


# Example usage
text = "今日は今日はとても良い日です。"
processed_text = preprocess_japanese_text(text)
print(processed_text)

今日 は 今日 は とても 良い 日 です ｡


In [5]:
import pandas as pd

df = pd.read_csv(
    "hf://datasets/Verah/tatoeba_dedupe_en-jp_2024-March-01/tatoeba_dedupe_random_en-jp_2024-03-01.tsv.gz",
    sep="\t",
)
df.to_csv("./datas/eng_jap_dataset.csv")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [6]:
df[:20]

,id,english,japanese
0,1,While in England I often consulted the guidebook.,イギリスにいる間、私はよくそのガイドブックを参考にした。
1,2,Look at the sports car over there.,あそこのスポーツカーを見なさい。
2,3,Never did I expect that he would fail the exam...,彼が試験に失敗するなんて私は予想もしなかった。
3,4,He knows no foreign language except English.,彼は英語以外の外国語は全く知らない。
4,5,All their secrets have been revealed.,彼らの秘密が全部暴かれた。
5,6,He promised to return the money without fail.,彼は間違いなく金を返すと約束した。
6,7,"To tell the truth, things at home haven't been...",実は家庭が上手くいってなくてさ・・・。離婚しようかと思ってるんだ。
7,8,Can I have your phone number?,電話番号教えてもらってもいい？
8,9,Even a monkey can solve a problem as simple as...,こんなに簡単な問題は猿さえも解けますよ。
9,10,You had better speak more naturally.,君はもっと自然に話す方がよい。


### Removing Emojis

In [7]:
import re
import pandas as pd
import tqdm


emoji_list_datas_path = "./datas//Emoji Sheets - Emoji Only.csv"
emoji_df = pd.read_csv(emoji_list_datas_path)

# Extract the emojis into a list
emoji_list = emoji_df["Emoji_List"].tolist()

# Start the pattern string
pattern = "["

# Append each code point to the pattern string, ensuring each one is 8 digits
for cp in emoji_list:
    pattern += f"\\U{cp[1:]:0>8}"

# Close the pattern string
pattern += "]"

# Compile the regular expression
emoji_pattern = re.compile(pattern, re.UNICODE)


# function to completely remove the emojis from the comments using re
def remove_emojis(text):
    if not isinstance(text, str):
        return text  # or return an empty string: return ''

    # Compile the regular expression
    emoji_pattern = re.compile(pattern, re.UNICODE)

    # Use the sub method to remove emojis
    text_no_emojis = emoji_pattern.sub(r"", text)
    return text_no_emojis

In [8]:
from tqdm import tqdm

tqdm.pandas(desc="Removing emojis from English text")
df["english"] = df["english"].progress_apply(remove_emojis)

tqdm.pandas(desc="Removing emojis from Japanese text")
df["japanese"] = df["japanese"].progress_apply(remove_emojis)

Removing emojis from Japanese text: 100%|██████████| 201607/201607 [00:08<00:00, 24064.60it/s]


## b

In [9]:
import mojimoji


def zen_to_han(text):
    return mojimoji.zen_to_han(text)

### Removing accented characters

In [10]:
import unicodedata


# English might have accent like é but Japanese doesn't have any accent I just create different function to ascii for Japanese and English
# Removing accented characters
def english_unicode_to_ascii(text):
    return "".join(
        ascii_text
        for ascii_text in unicodedata.normalize("NFKD", text)
        .encode("ascii", "ignore")
        .decode("utf-8", "ignore")
    )


def japanese_unicode_to_ascii(text):
    return " ".join(ascii_text for ascii_text in unicodedata.normalize("NFKD", text))


japanese_unicode_to_ascii("こんにちは。 今日は"), english_unicode_to_ascii(
    "Hello world é "
)

('こ ん に ち は 。   今 日 は', 'Hello world e ')

## Remove all punctuation and special characters, keeping only the specified characters (a-z, A-Z, Kanji, Katakana, Hiragana)

In [11]:
import re


def clean_text(text):
    # Define regex pattern for allowed characters (Latin letters and Japanese characters)
    allowed_pattern = r"[^a-zA-Z\u4E00-\u9FFF\u3040-\u30FF\s]"
    # Replace everything not in allowed pattern with a space
    cleaned_text = re.sub(allowed_pattern, " ", text)

    # Remove extra spaces
    cleaned_text = re.sub(r"\s+", " ", cleaned_text).strip()

    cleaned_text = cleaned_text.lower()

    return cleaned_text


# Example usage
text = "こんにちは、世界! This is an example sentence."
print(clean_text(text))

こんにちは 世界 this is an example sentence


In [12]:
from janome.tokenizer import Tokenizer
import mojimoji

# Initialize Japanese tokenizer
tokenizer = Tokenizer()


def preprocess_japanese_text(text):

    # TODO: Uncomment the line below to convert Japanese text to ASCII characters PROBLEM: cause the problem in tokenization
    # text=japanese_unicode_to_ascii(text)

    # Convert full-width characters to half-width
    text = mojimoji.zen_to_han(text)

    text = clean_text(text)

    # Tokenize the text
    tokens = tokenizer.tokenize(text, wakati=True)

    # Join tokens with a space to preserve word boundaries for translation models
    preprocessed_text = " ".join(tokens)

    # Remove any unnecessary whitespace
    preprocessed_text = preprocessed_text.strip()

    return "start_ " + preprocessed_text + " _end"


text = "猫と犬がけんかしている。"
processed_text = preprocess_japanese_text(text)
print(processed_text)

start_ 猫 と 犬 が けんか し て いる _end


In [13]:
from transformers import BertTokenizer

def preprocess_english_text(text):
    # Initialize the tokenizer
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

    # Normalize text
    text = text.lower()

    text=english_unicode_to_ascii(text)

    text=clean_text(text)

    # Tokenize text
    tokens = tokenizer.tokenize(text)

    # Join tokens with a space to preserve word boundaries for translation models
    preprocessed_text = " ".join(tokens)

    # Remove any unnecessary whitespace
    preprocessed_text = preprocessed_text.strip()

    return "start_ " + preprocessed_text + " _end"

    # # Convert tokens to token IDs
    # token_ids = tokenizer.convert_tokens_to_ids(tokens)

    # # Add special tokens
    # token_ids = tokenizer.build_inputs_with_special_tokens(token_ids)

    # # Create attention mask
    # attention_mask = [1] * len(token_ids)

    # return token_ids, attention_mask

# Example usage
text = "Hello, how are you?"
token = preprocess_english_text(text)
token

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


'start_ hello how are you _end'

In [14]:
# Sample Japanese sentence
sentence = "私は猫が好きです。"

# Tokenize and print POS information
for token in tokenizer.tokenize(sentence):
    print(f"Word: {token.surface}, POS: {token.part_of_speech}")

Word: 私, POS: 名詞,代名詞,一般,*
Word: は, POS: 助詞,係助詞,*,*
Word: 猫, POS: 名詞,一般,*,*
Word: が, POS: 助詞,格助詞,一般,*
Word: 好き, POS: 名詞,形容動詞語幹,*,*
Word: です, POS: 助動詞,*,*,*
Word: 。, POS: 記号,句点,*,*


In [ ]:
from tqdm  import tqdm


tqdm.pandas(desc="Preprocessing English text")
df["english_preprocessed"] = df["english"][:10000].progress_apply(preprocess_english_text)


tqdm.pandas(desc="Preprocessing Japanese text")
df["japanese_preprocessed"] = df["japanese"][:10000].progress_apply(preprocess_japanese_text)

Preprocessing English text:   0%|          | 0/10000 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Preprocessing Japanese text:  52%|█████▏    | 5205/10000 [00:08<00:09, 481.88it/s]

In [ ]:
df.to_csv("./datas/eng_jap_dataset_preprocessed.csv")